# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v1', file_sigmas ="./svmCodeSVD/sigmas1.dat" )

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)

###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas1.dat


## Twin Delayed DDPG (TD3) from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.2 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise = action_noise, \
            learning_starts=150, batch_size=150, gamma=1.0, verbose=1, seed = 2)

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=500, log_interval = 5, n_eval_episodes = 1)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## PPO with GAE from `stable_baseline3` 

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

model = PPO("MlpPolicy", env, verbose=1, n_steps = 2, batch_size = 64, gamma = 1.0)

# classstable_baselines3.ppo.PPO(policy, env, learning_rate=0.0003, n_steps=2048, 
#         batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
#         clip_range_vf=None, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, 
#         use_sde=False, sde_sample_freq=- 1, target_kl=None, tensorboard_log=None, 
#         create_eval_env=False, policy_kwargs=None, verbose=0, seed=None, device='auto', 
#         _init_setup_model=True)

model.learn(total_timesteps = 350, n_eval_episodes = 1)

# learn(total_timesteps, callback=None, log_interval=1, eval_env=None, eval_freq=- 1, 
#       n_eval_episodes=5, tb_log_name='PPO', eval_log_path=None, reset_num_timesteps=True)

model.save("ppo_svm")

In [ ]:
model = PPO.load("ppo_svm")

obs = env.reset()
rewards = []
score = 0.0
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    

## From my `ddpg_agent.py` code

In [3]:
from ddpg_agent import Agent
agent = Agent(1, 3, random_seed=2)

In [4]:
## Save all rewards, energies and princip dims in files during training

def save_all(agent, rewards, energies, princip_dims):
    torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
    torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

    name_rewards = 'rewards_RL_0.out'
    file_rewards = open(name_rewards,'w')
    np.savetxt(file_rewards, rewards, fmt="%f")
    file_rewards.close()

    name_energies = 'energies_RL_0.out'
    file_energies = open(name_energies,'w')
    np.savetxt(file_energies, energies, fmt="%f")
    file_energies.close()

    name_dim = 'princip_dims_RL_0.out'
    file_dim = open(name_dim,'w')
    np.savetxt(file_dim, princip_dims, fmt="%f")
    file_dim.close()

def run_ddpg(max_t_step = 300, n_episodes=700):
    ##Inizialization
    rewards = []
    energies = []  
    princip_dims = []

    for i_episode in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_per_i_episode = []
        energies_per_i_episode = []
        princip_dim_per_i_episode = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save
            rew_per_i_episode.append(reward)
            energies_per_i_episode.append(state[0])
            princip_dim_per_i_episode.append(env.princp_dim)
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        rewards.append(rew_per_i_episode)
        energies.append(energies_per_i_episode)
        princip_dims.append(princip_dim_per_i_episode)

        save_all(agent, rewards, energies, princip_dims)

        print('Episode {} ... Score: {:.3f}'.format(i_episode, np.sum(rewards[i_episode])))

        
    return rewards, energies, princip_dims


In [ ]:
all_rewards, all_energies, all_princip_dim = run_ddpg()

#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [36.242744 48.282074 51.0255  ]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0274688
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.370702015662202
#### CALL STEP #### 2
Action chosen at step:  [28.180555 46.97483  61.97136 ]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0187526
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.764582143013298
#### CALL STEP #### 3
Action chosen at step:  [45.071487 62.3534   99.84169 ]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.00629539
With this action the full dim is:  3  and princip dim is:  3
###

With this action the energy is:  -0.00778749
With this action the full dim is:  25  and princip dim is:  25
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.08100147953901171
#### CALL STEP #### 26
Action chosen at step:  [ 57.30811   63.368137 100.421486]
Basis size (it should be the same of full dim) =   26
With this action the energy is:  -0.00790771
With this action the full dim is:  26  and princip dim is:  26
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.08936151385326063
#### CALL STEP #### 27
Action chosen at step:  [ 52.38388   61.459126 110.      ]
Basis size (it should be the same of full dim) =   27
With this action the energy is:  -0.00832902
With this action the full dim is:  27  and princip dim is:  27
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.11865918526347308
#### CALL STEP #### 28
Action chosen at step:  [ 39.17717   41.188522 110.      ]
Basis size (it should be the same of full dim)

With this action the energy is:  -0.0844007
With this action the full dim is:  46  and princip dim is:  46
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  5.408643007776511
#### CALL STEP #### 55
Action chosen at step:  [57.232204 76.42425   3.586071]
Basis size (it should be the same of full dim) =   47
With this action the energy is:  -0.0845126
With this action the full dim is:  47  and princip dim is:  47
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  5.4164244737535885
#### CALL STEP #### 56
Action chosen at step:  [67.70469 77.47153  0.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP #### 57
Action chosen at step:  [84.68733  68.408905  0.      ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP #### 58
Action chosen at step:  [76.21104  50.67

With this action the energy is:  -0.088751
With this action the full dim is:  68  and princip dim is:  67
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  5.711160536284866
#### CALL STEP #### 83
Action chosen at step:  [69.45013 54.84677  0.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP #### 84
Action chosen at step:  [58.68162  54.116302 11.875076]
Basis size (it should be the same of full dim) =   69
With this action the energy is:  -0.088816
With this action the full dim is:  69  and princip dim is:  68
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  5.715680601419005
#### CALL STEP #### 85
Action chosen at step:  [82.80615  45.434414  9.045956]
Basis size (it should be the same of full dim) =   70
With this action the energy is:  -0.0891793
With this action the full dim is:  70  and princip dim is:  69
#### THE ACTION IS A GOOD ONE ##

With this action the energy is:  -0.0941029
With this action the full dim is:  91  and princip dim is:  83
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  6.083328791537117
#### CALL STEP #### 109
Action chosen at step:  [110.        46.18246   20.461132]
Basis size (it should be the same of full dim) =   92
With this action the energy is:  -0.094149
With this action the full dim is:  92  and princip dim is:  84
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  6.086534560809174
#### CALL STEP #### 110
Action chosen at step:  [110.        85.228714  40.862106]
Basis size (it should be the same of full dim) =   93
With this action the energy is:  -0.0941587
With this action the full dim is:  93  and princip dim is:  84
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  6.0872090936061145
#### CALL STEP #### 111
Action chosen at step:  [110.        67.1076    42.311523]
Basis size (it should be the same of full dim) =   9

...

With this action the energy is:  -0.00680421
With this action the full dim is:  110  and princip dim is:  40
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.012624715768307482
#### CALL STEP #### 111
Action chosen at step:  [ 44.906494 110.        47.636154]
Basis size (it should be the same of full dim) =   111
With this action the energy is:  -0.00754573
With this action the full dim is:  111  and princip dim is:  41
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.06418961881855978
#### CALL STEP #### 112
Action chosen at step:  [ 48.726418 110.        57.339844]
Basis size (it should be the same of full dim) =   112
With this action the energy is:  -0.00830827
With this action the full dim is:  112  and princip dim is:  41
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.11721624139373077
#### CALL STEP #### 113
Action chosen at step:  [ 55.606777 110.        78.1662  ]
Basis size (it should be the same of 

With this action the energy is:  -0.00912635
With this action the full dim is:  135  and princip dim is:  43
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.17410508577736117
#### CALL STEP #### 136
Action chosen at step:  [110.        58.267403 110.      ]
Basis size (it should be the same of full dim) =   136
With this action the energy is:  -0.00912338
With this action the full dim is:  136  and princip dim is:  43
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.17389855357046535
#### CALL STEP #### 137
Action chosen at step:  [110.        54.439632 105.83801 ]
Basis size (it should be the same of full dim) =   137
With this action the energy is:  -0.00913299
With this action the full dim is:  137  and princip dim is:  43
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.17456682781568134
#### CALL STEP #### 138
Action chosen at step:  [110.       50.9036   97.80652]
Basis size (it should be the same of full

With this action the energy is:  -0.00912603
With this action the full dim is:  160  and princip dim is:  45
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.17408283314900963
#### CALL STEP #### 161
Action chosen at step:  [100.10354 110.      110.     ]
Basis size (it should be the same of full dim) =   161
With this action the energy is:  -0.0091246
With this action the full dim is:  161  and princip dim is:  45
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.17398339171605848
#### CALL STEP #### 162
Action chosen at step:  [104.65741 110.       91.40523]
Basis size (it should be the same of full dim) =   162
With this action the energy is:  -0.00912566
With this action the full dim is:  162  and princip dim is:  45
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.1740571035474776
#### CALL STEP #### 163
Action chosen at step:  [110.        78.47687   81.799255]
Basis size (it should be the same of full dim)

With this action the energy is:  -0.00895202
With this action the full dim is:  185  and princip dim is:  46
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.161982271087604
#### CALL STEP #### 186
Action chosen at step:  [110.      110.       94.29346]
Basis size (it should be the same of full dim) =   186
With this action the energy is:  -0.00895202
With this action the full dim is:  186  and princip dim is:  46
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.161982271087604
#### CALL STEP #### 187
Action chosen at step:  [110.      110.       64.54889]
Basis size (it should be the same of full dim) =   187
With this action the energy is:  -0.00895839
With this action the full dim is:  187  and princip dim is:  46
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.16242523747074777
#### CALL STEP #### 188
Action chosen at step:  [110. 110. 110.]
Basis size (it should be the same of full dim) =   188
With this a

With this action the energy is:  -0.00901366
With this action the full dim is:  210  and princip dim is:  47
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.1662686836240379
#### CALL STEP #### 211
Action chosen at step:  [ 94.585236  84.10828  110.      ]
Basis size (it should be the same of full dim) =   211
With this action the energy is:  -0.00901422
With this action the full dim is:  211  and princip dim is:  47
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.16630762572365398
#### CALL STEP #### 212
Action chosen at step:  [ 94.8479    91.426216 103.667786]
Basis size (it should be the same of full dim) =   212
With this action the energy is:  -0.00898946
With this action the full dim is:  212  and princip dim is:  47
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.16458582860486715
#### CALL STEP #### 213
Action chosen at step:  [ 82.21376  84.65474 100.34808]
Basis size (it should be the same of full 

In [ ]:
import matplotlib.pyplot as plt

scores = np.loadtxt('scores_RL.out')
energies = np.loadtxt('energies_RL.out')
dim = np.loadtxt('princip_dim_RL.out')

plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(energies)), energies)
plt.ylabel('Eenergies (mK)')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(dim)), dim)
plt.ylabel('dim (mK)')
plt.xlabel('Episode #')

In [ ]:
rewards = []

for i in range(10):
    rew_i_episode = []
    for j in range(20):
        rew_i_episode.append(j)
    rewards.append(rew_i_episode)
    print('Episode {} ... Score: {:.3f}'.format(i, np.sum(rewards[i])))
    
name_rewards = 'rewards_RL.out'
file_rewards = open(name_rewards,'w')
np.savetxt(file_rewards, rewards, fmt="%f")
file_rewards.close()

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break